# Setup

additional information: https://github.com/niessner/Matterport/blob/master/data_organization.md 

In [ ]:
import sys
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
# subprocess.check_call([sys.executable, "-m", "pip", "install", "open3d"])
# subprocess.check_call([sys.executable, "-m", "pip", "install", "trimesh"])
# subprocess.check_call([sys.executable, "-m", "pip", "install", "imageio"])
# subprocess.check_call([sys.executable, "-m", "pip", "install", "pillow"])
# subprocess.check_call([sys.executable, "-m", "pip", "install", "tqdm"])


### File Locations

In [ ]:
camera_transforms_path = r"G:/My Drive/GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/undistorted_camera_parameters/17DRP5sb8fy/undistorted_camera_parameters/17DRP5sb8fy.conf"
meshes_path = r"G:/My Drive/GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/matterport_mesh/17DRP5sb8fy/_/matterport_mesh/bed1a77d92d64f5cbbaaae4feed64ec1/bed1a77d92d64f5cbbaaae4feed64ec1.obj"
depth_imgs_dir = r"G:/My Drive//GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/undistorted_depth_images/17DRP5sb8fy/_/undistorted_depth_images"
color_imgs_dir = r"G:/My Drive//GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/undistorted_color_images/17DRP5sb8fy/_/undistorted_color_images"
mask_imgs_dir = r"G:/My Drive/GSD_AI/05_Segmentation/17DRP5sb8fy/SAM"
segmentation_data_csv_path = r"G:/My Drive/GSD_AI/05_Segmentation/17DRP5sb8fy/SAM/ImageClassifications_collected.csv"
model_output_path = r"C:/Repos/GL-Material-Classification/dbg.obj"

### Settings

In [ ]:
from enum import Enum
class CategoryType(Enum):
    FAMILY = 2
    CATEGORY = 3
    MATERIAL = 4
    
img_width = 1280
img_height = 1024
scale_factor = 4000
img_sample_stride = 10 #density of samples of depth maps, denser=slower but more accurate
dist_threshold = 0.5 #max dist of raycast point to mesh
category_colors = {'unknown': '#000000'} # categories that mesh faces can go into, starting with a special category of "unknown", which is used when a face has no classifications applied to it
category_type = CategoryType.MATERIAL # the column from the CSV to extract for categorization

### Load Data

In [ ]:
import open3d as o3d
import open3d.core as o3c
import trimesh
from open3d.t.geometry import RaycastingScene
import numpy as np
import os
import re
import random
from PIL import ImageColor
import imageio.v3 as iio
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor
from collections import defaultdict


In [ ]:
raw_scan = o3d.io.read_triangle_mesh(meshes_path)
raw_scan.paint_uniform_color((0.25,0.25,0.25))
#setup raycast scene
ray_cast_scene = o3d.t.geometry.RaycastingScene()
geom_id_to_mesh = {}

full_mesh = o3d.t.geometry.TriangleMesh.from_legacy(raw_scan)
geom_id = ray_cast_scene.add_triangles(full_mesh)
print(geom_id)

#geom_id_to_mesh[geom_id] = geom_id

### Generate Materials Dictionary, Parse pregenerated CSV

In [ ]:
def generate_random_hex_color():
    """Generates a random hexadecimal color code."""
    # Generate random integer values for red, green, and blue components (0-255)
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)

    # Format the RGB values into a 6-digit hexadecimal string
    # :02x ensures each component is formatted as a two-digit hexadecimal number,
    # padding with a leading zero if necessary.
    hex_color = f"#{r:02x}{g:02x}{b:02x}"
    return hex_color

class MaskImage:
   key = ''
   mask_path = ''
   categorization = ''

   def __init__(self, key, path, cat):
      self.key = key
      self.mask_path = path
      self.categorization = cat
      return

mask_imgs = []

with open(segmentation_data_csv_path, 'r') as file:
   lines = file.readlines()
   ct = len(lines)
   pbar = tqdm(total=ct)
   for i, line in enumerate(lines):
      line = line.strip()
      sections = line.split(',')
      orig_path = sections[0]
      #desc = sections[1]
      # fam = sections[2]
      # cat = sections[3]
      # mat = sections[4]
      cat = sections[category_type.value]
      cat = cat.strip()
      if cat == '':
         print("skipping image, no category provided")
         continue
      if not cat in category_colors:
         category_colors[cat] = generate_random_hex_color()
      
      sections = re.split(r'[/.]', orig_path)
      mask_id = sections[len(sections) - 2]
      key = sections[len(sections) - 4]
      mask_path = f"{mask_imgs_dir}/{key}/masks/{mask_id}.png"
      if not os.path.exists(mask_path):
        print(f"mask image not found: {mask_path}")
        continue
      mask_imgs.append(MaskImage(key, mask_path, cat))
      pbar.update(1)
   pbar.close()
   print(f"processed {ct} lines of CSV")



#### Check That Everything Imported
Note that this will not work in Colab, only in jupyter notebooks running on your own setup.

In [ ]:
# o3d.visualization.draw_geometries([raw_scan])

#### Load Camera Transforms and Data

In [ ]:
class camera_pose_data:

  camera_transform = np.empty(0)
  depth_img = ""
  color_img = ""
  focal_len_x_pixels, focal_len_y_pixels, camera_center_x_pixels, camera_center_y_pixels = 0, 0, 0, 0
  camera_position = np.array([[0.0, 0.0, 0.0]])
  mask_imgs = []
  def __init__(self, depth_img, color_img, camera_transform, focal_len_x_pixels, focal_len_y_pixels, camera_center_x_pixels, camera_center_y_pixels):
    self.depth_img = depth_img
    self.color_img = color_img
    self.camera_transform = camera_transform
    self.focal_len_x_pixels = focal_len_x_pixels
    self.focal_len_y_pixels = focal_len_y_pixels
    self.camera_center_x_pixels = camera_center_x_pixels
    self.camera_center_y_pixels = camera_center_y_pixels
    return

camera_poses = {}
try:
  with open(camera_transforms_path, 'r') as file:
    focal_len_x_pixels, focal_len_y_pixels, camera_center_x_pixels, camera_center_y_pixels = 0, 0, 0, 0
    for i, line in enumerate(file):
      clean_line = line.strip()
      sections = clean_line.split(' ')
      cleaned_sections = []
      for s in sections:
        if (s != ''):
          cleaned_sections.append(s)
      sections = cleaned_sections

      ct = len(sections)
      if (ct < 10):
        continue
      if (sections[0] == "intrinsics_matrix" and ct == 10):
        focal_len_x_pixels = float(sections[1])
        focal_len_y_pixels = float(sections[5])
        camera_center_x_pixels = float(sections[3])
        camera_center_y_pixels = float(sections[6])
      if (sections[0] != "scan"):
        continue
      if (ct != 19):
        print(f"ERROR: Unexpected number of sections in line {i}")
      depth_img_path = depth_imgs_dir + "/" + sections[1]
      rgb_img_path = color_imgs_dir + "/" + sections[2]

      if (not os.path.exists(depth_img_path) or not os.path.exists(rgb_img_path)):
        print(f"ERROR: File not found: {depth_img_path} {rgb_img_path}")
        continue
      t = np.eye(4)#empty 4x4 transform matrix
      for r in range(0, 4):
        for c in range (0, 4):
          ray_idx = 3 + (r * 4) + c
          t[r, c] = float(sections[ray_idx])

      #rotate 180degrees around x axis--it was found that doing this extra transform was necessary rather than relying on the imported transform by itself
      #upper left 3x3 part of the 4x4 transform matrix is the rotation matrix
      #this gives us rotate_x = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]
      #matrix for rotating about x-axis: Rx(t)= [[1, 0, 0], [0, cos(t), -sin(t)], [0, sin(t), cos(t)]]
      rotate_x = np.eye(4)
      rotate_x[1,1] = -1
      rotate_x[2,2] = -1
      t = t @ rotate_x

      pose = camera_pose_data(depth_img_path, rgb_img_path, t, focal_len_x_pixels, focal_len_y_pixels, camera_center_x_pixels, camera_center_y_pixels)

      #get image key
      sections = re.split(r'[/.]', rgb_img_path)
      key = sections[len(sections) - 2]
      #print(f"image name: {key}")
      if not key in camera_poses:
        camera_poses[key] = pose
      else:
        print(f"key already found in dictionary: {key}")
      




except FileNotFoundError:
  print(f"File not found: {camera_transforms_path}")

print(f"successfully imported {len(camera_poses)} poses")

In [ ]:
print(camera_poses[next(iter(camera_poses))].camera_transform) #check that transform values came in properly

# Process Depth Images

In [ ]:
#check that file loads and has a valid type
#images are uint16 pngs where each integer increment = 0.25mm (/4000 to get meters)

img = o3d.io.read_image(camera_poses[next(iter(camera_poses))].depth_img)
print("dtype:", np.asarray(img).dtype)
print("min/max:", np.min(np.asarray(img)), np.max(np.asarray(img)))


In [ ]:
def depth_to_point_cloud(depth_img, stride, intrinsic, scale):
    depth_np = np.asarray(depth_img).astype(np.float32) / float(scale)
    h, w = depth_np.shape
    fx, fy = intrinsic.get_focal_length()
    cx, cy = intrinsic.get_principal_point()

    # Create meshgrid of pixel coordinates
    u, v = np.meshgrid(np.arange(0, w, stride), np.arange(0, h, stride))
    z = depth_np[::stride, ::stride]
    x = (u - cx) * z / fx
    y = (v - cy) * z / fy

    points = np.stack((x, y, z), axis=-1).reshape(-1, 3)
    pixel_coords = np.stack((u, v), axis=-1).reshape(-1, 2)
    validity_map = []
    for p in points:
      if (p[0] == 0 and p[1] == 0  and p[2] == 0 ):
        validity_map.append(False)
      else:
        validity_map.append(True)
    points = points[validity_map]

    return points, validity_map, pixel_coords

depth_clouds = {}
validity_maps = {}
pixel_maps = {}
camera_positions = []
ct = len(camera_poses)
i = 0
pbar = tqdm(total = ct)
for key in camera_poses:
  pose = camera_poses[key]
  img = o3d.io.read_image(pose.depth_img)
  #print(pose.depth_img)
  #img = iio.imread(pose.depth_img)
  intrinsic = o3d.camera.PinholeCameraIntrinsic(width = img_width, height = img_height, fx = pose.focal_len_x_pixels, fy = pose.focal_len_y_pixels, cx = pose.camera_center_x_pixels, cy = pose.camera_center_y_pixels)
  #o3d was crashing when applying project_valid_depth_only = False and stride > 1. We need information for each sample, not just valid ones.
  cloud_pts, validity_map, pixel_coords = depth_to_point_cloud(img, img_sample_stride, intrinsic, scale_factor)
  validity_maps[key] = validity_map
  pixel_maps[key] = pixel_coords
  cloud = o3d.geometry.PointCloud()
  cloud.points = o3d.utility.Vector3dVector(cloud_pts)

  # don't do this:
  # cloud = o3d.geometry.PointCloud.create_from_depth_image(img,
  #                                                               intrinsic,
  #                                                               depth_scale = scale_factor,
  #                                                               depth_trunc = 50,
  #                                                               stride = img_sample_stride,
  #                                                               project_valid_depth_only = False)

  # Get Camera Position transformed
  camera_position = cam_pos_world = pose.camera_transform[:3, 3]
  pose.camera_position = camera_position
  camera_positions.append(camera_position)

  #transform the depth cloud
  cloud.transform(pose.camera_transform)
  depth_clouds[key] = cloud
  # print(f"cloud size: {len(cloud.points)}")
  # print(f"{str(i + 1)} of {str(ct)}")
  pbar.update(1)
  i += 1
  # if (i > 50):
  #   break

pbar.close()


## Verify depth clouds match model.
You can look at a specific image and visually-inspect if the point cloud is aligning properly with the model. Some minor deviation and random noise is expected.

In [ ]:
# clds = []
# for cloud in depth_clouds:
#     clds.append(depth_clouds[cloud])
# o3d.visualization.draw_geometries([raw_scan, clds[20]])

# Denoise Point Clouds

Check each point cloud point and make sure it is close to mesh. If not, remove it.

In [ ]:
def euclidean_distance(point1, point2):
    "Calculates the Euclidean distance between two 3D points."
    point1 = np.asarray(point1)
    point2 = np.asarray(point2)
    return np.linalg.norm(point1 - point2)


face_id_maps = {}
#geometry_id_maps = {}

ct = len(depth_clouds)
mod_validity_maps = validity_maps.copy()
cloudpts = []
i = -1
pbar = tqdm(total=ct)
for key in depth_clouds:
  i += 1
  
  cloud = depth_clouds[key]
  v_map = mod_validity_maps[key]
  pose = camera_poses[key]

  ray_sources = []
  ray_targets = []
  map_size = len(v_map)
  local_valid_map = v_map.copy()
  local_triangle_ids = np.full(map_size, -1)

  #get ray sources/targets
  #this needs to be limited to valid parts of the depth image, to get a ray that is valid
  position = pose.camera_position
  valid_ct = 0
  #print(f"pt ct: {len(cloud.points)}")
  cloudIdx = 0
  for ray_idx, v in enumerate(local_valid_map):
    if not local_valid_map[ray_idx]:
      continue
    point = cloud.points[cloudIdx]
    cloudIdx += 1
    ray_sources.append(position)
    ray_targets.append(point)
    cloudpts.append(point)
    valid_ct += 1
  #print(f"starting valid ct: {valid_ct}")

  #normalize the rays
  #print(f"ray ct: {len(ray_sources)}")
  rays_sources = np.array(ray_sources)
  rays_targets = np.array(ray_targets)
  directions = ray_targets - rays_sources
  norms = np.linalg.norm(directions, axis=1, keepdims= True)
  directions_normalized = directions / norms
  rays = o3c.Tensor(np.hstack((rays_sources, directions_normalized)), dtype=o3c.Dtype.Float32)

  #compute
  results = ray_cast_scene.cast_rays(rays)
  hit_points = ray_sources + directions_normalized * results["t_hit"].numpy()[..., None]

  #bring values from shortened "valid" ray list back to the full array
  map_idx = 0
  for ray_idx, ids in enumerate(results['primitive_ids']):
    id = ids.item()
    if  id == RaycastingScene.INVALID_ID:
      local_valid_map[map_idx] = False
      continue
    hp = hit_points[ray_idx]
    dist = euclidean_distance(ray_targets[ray_idx], hp)
    if (dist > dist_threshold):
      local_valid_map[map_idx] = False
      continue


    local_valid_map[map_idx] = True
    local_triangle_ids[map_idx] = id
    map_idx += 1

  face_id_maps[key] = local_triangle_ids

  mod_validity_maps[key] = local_valid_map
  pbar.update(1)
  # print(f"ending valid ct: {valid_ct}")
  # print(len(results['primitive_ids']))
  # print(f"{str(i + 1)} of {str(ct)}")

pbar.close()





# Apply Segmentation Results to Model

In [ ]:
face_id_categories = np.zeros((len(full_mesh.triangle.indices), len(category_colors)), dtype=np.int32)
print(f"size of face id to categories array: {face_id_categories.shape}")
mask_ct = 0
cat_list = list(category_colors)

def process_mask(m_img):
    key = m_img.key
    path = m_img.mask_path
    cat = m_img.categorization

    try:
        #pose = camera_poses[key]
        v_map = validity_maps[key]
        pixel_coords = pixel_maps[key]
        face_ids = face_id_maps[key]
    except KeyError:
        print(f"{key} not found in dictionary. Are you processing a subset of frames?")
        return None

    cat_id = cat_list.index(cat)
    img = iio.imread(path)

    valid_indices = np.where(v_map)[0]
    coords = pixel_coords[valid_indices]
    face_ids_valid = face_ids[valid_indices]
    pix_vals = img[coords[:, 1], coords[:, 0]]
    mask_hits = (pix_vals != 0)
    selected_face_ids = face_ids_valid[mask_hits]

    # Accumulate counts locally
    local_counts = defaultdict(int)
    for fid in selected_face_ids:
        local_counts[fid] += 1

    return (cat_id, local_counts)

# Multithreaded execution
mask_ct = 0
pbar = tqdm(total=len(mask_imgs))
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_mask, m_img) for m_img in mask_imgs]
    for future in futures:
        result = future.result()
        if result is None:
            continue
        cat_id, local_counts = result
        for fid, count in local_counts.items():
            face_id_categories[fid, cat_id] += count
        mask_ct += 1
        #print(f"Processed {mask_ct} of {len(mask_imgs)}")
        pbar.update(1)

pbar.close()
print(f"successfully processed {mask_ct} masks")


## Decide vote winners
For each mesh face, the material with the most votes wins.

In [ ]:
winners = []
colors = []
mesh_faces_by_category = {}
for cat in cat_list:
    mesh_faces_by_category[cat] = []
for i, face_tallies in enumerate(face_id_categories):
    #print(str(face_tallies))
    if np.max(face_tallies) == 0:
        max_idx = 0
    else:
        max_idx = np.argmax(face_tallies)
    #print(str(max_idx))
    w = cat_list[max_idx]
    winners.append(w)
    mesh_faces_by_category[w].append(i)
    colors.append(np.array(ImageColor.getcolor(category_colors[w], "RGB"))/255.0)
print("winners determined")

# Visualize

## Split Mesh by Category

In [ ]:
triangles = full_mesh.triangle["indices"].numpy()
vertices = full_mesh.vertex["positions"].numpy()
meshes_by_category = {}
for key in mesh_faces_by_category:
    face_ids = mesh_faces_by_category[key]
    if len(face_ids) == 0:
        continue
    # Get triangle data (array shape (N,3)), vertex indices for each triangle
    selected_triangles = triangles[face_ids]
    # Get unique vertices
    unique_vertex_indices = np.unique(selected_triangles)
    # make a dictionary that maps the old vertex indices to new ones for the new mesh that will be created
    old_to_new = {old_idx: new_idx for new_idx, old_idx in enumerate(unique_vertex_indices)} # eg old_to_new = {7: 0, 19: 1, 42: 2}
    remapped_triangles = np.vectorize(old_to_new.get)(selected_triangles) # eg np.vectorize(old_to_new.get)([42, 7, 19]) → [2, 0, 1], creates a function that then you call with the param of selected_triangles
    # extract vertex positions
    new_vertices = vertices[unique_vertex_indices]
    # create new tensor mesh
    new_mesh = o3d.geometry.TriangleMesh()
    new_mesh.vertices = o3d.utility.Vector3dVector(new_vertices)
    new_mesh.triangles = o3d.utility.Vector3iVector(remapped_triangles)

    #color by material
    c = np.array(ImageColor.getcolor(category_colors[key], "RGB"))/255.0
    new_mesh.paint_uniform_color(c)

    # store mesh in dictionary
    meshes_by_category[key] = new_mesh


## Visualize Results

In [ ]:
# Step 3: Visualize
preview_mesh = meshes_by_category['wood']
o3d.visualization.draw_geometries(
    [preview_mesh],
    zoom=0.7,
    front=[0, 0, -1],
    lookat=preview_mesh.get_center(),
    up=[0, -1, 0]
)

## Export Meshes

In [ ]:
def o3d_to_trimesh(o3d_mesh):
    vertices = np.asarray(o3d_mesh.vertices)
    faces = np.asarray(o3d_mesh.triangles)
    return trimesh.Trimesh(vertices=vertices, faces=faces)

trimesh_scene = trimesh.Scene()
for key in meshes_by_category:
    t_mesh = o3d_to_trimesh(meshes_by_category[key])
    trimesh_scene.add_geometry(t_mesh, geom_name=key)

trimesh_scene.export(model_output_path)